In [1]:
def signals1(Noversampl, depth0, oof, ofcont, FILT, Order, FiltType, FILTFACT, AutoPhase, ph0, IQ, noise, SNR):
    
    sstep = nptslambda/Noversampl
    depth = depth0 * 1e-6
    ph0 = ph0/2/np.pi
    ph = int(nptslambda * ph0)
    
    #FRINGE
    er = np.sqrt(amp) #reference field
    es = R*np.sqrt(amp)*np.exp(1j*(2*np.pi*oof*(2*depth/c))) #sample field
    
    #I/Q detection and background subtraction
    if BG:
        sfI = np.square(np.abs(er + es)) - (np.square(er) + np.square(R*np.sqrt(amp)))
        sfQ = np.square(np.abs(er + es*np.exp(1j*np.pi/2)))  - (np.square(er) + np.square(R*np.sqrt(amp)))
    else:
        sfI = np.square(np.abs(er + es))
        sfQ = np.square(np.abs(er + es*np.exp(1j*np.pi/2)))
    
    #add noise
    sfI = sfI + np.random.randint(-10000,10000,len(oof))/10000*noise
    sfQ = sfQ + np.random.randint(-10000,10000,len(oof))/10000*noise

    if IQ:
        sf = sfI + 1j*sfQ
    else:
        sf = sfI    
    
    #SNR
    if SNR:
        sfI_snr = np.square(np.abs(er)) + np.random.randint(-10000,10000,len(oof))/10000*noise - np.square(er) 
        sfQ_snr = np.square(np.abs(er)) + np.random.randint(-10000,10000,len(oof))/10000*noise - np.square(er)
        sf_snr = sfI_snr + 1j*sfQ_snr
                
    #ANALOG FILTER
    if FILT:
        if FiltType == 1:
            b,a = signal.butter(Order, FILTFACT/nptslambda, 'low') 
        else:
            b,a = signal.cheby1(Order,0.5,FILTFACT/nptslambda,'low')

        sfilt = signal.filtfilt(b,a,sf);
        
        if SNR:
            sfilt_snr = signal.filtfilt(b,a,sf_snr)
    else:
        sfilt = sf
        
        if SNR:
            sfilt_snr = sf_snr
        
    #SAMPLING
    #cropping the middle one of three sweep
    ind = int(len(amp)/3)
    amp0 = amp[ind+1:ind+1+npts]; 

    oof = oof[ind+1:ind+1+npts];
    ofcont = ofcont[ind+1:ind+1+npts];

    sf = sf[ind+1:ind+1+npts];

    #get acquisition phase
    if AutoPhase: 
        maxsearch = np.zeros(50)
        count_test = 0
        phase_test = np.linspace(0,0.98,50)
        for n in range(0,50):
            count_test = count_test + 1

            ph_test = int(nptslambda*phase_test[n])

            inds = ind+1+ph_test
            inde = ind+1+npts+ph_test
            indstep = int(sstep)
            s_test = sfilt[inds:inde:indstep]

            S_test = np.abs(np.fft.fftshift(np.fft.ifft(s_test,nfft))) 
            mag = max(S_test)
            maxsearch[n] = mag

        maxind = np.argmax(maxsearch)
        ph = int(nptslambda*phase_test[maxind])

    #inds = int(ind+1+ph)
    #inde = int(ind+1+npts+ph)
    #indstep = int(sstep)
    #ss = sfilt[inds:inde:indstep]
    sfilt_roll = np.roll(sfilt,ph)  #apply clock phase 
    
    inds = int(ind+1)
    inde = int(ind+1+npts)
    indstep = int(sstep)
    ss = sfilt_roll[inds:inde:indstep] # DAQ sampling
    ss = np.roll(ss,1) # To make fringe signal before and after sampling align in plot. Reduces confusion. Doesn't change anything. 
    
    sfilt_plot = sfilt[ind+1:ind+1+npts] # just for plotting fringe before sampling
    
    if SNR:
        ss_snr = sfilt_snr[inds:inde:indstep] 
    
    #DEPTH VECTOR
    inds = 0
    inde = int(len(ofcont))
    indstep = int(sstep)
    ofcont_sampled = ofcont[inds:inde:indstep]            #%sampled frequency vector
    df_sampled = ofcont_sampled[1]-ofcont_sampled[0]      #frequency resolution of sampled field
    dt_sampled = 1/(df_sampled*nfft);
    temp_vec = np.linspace(-nfft/2,nfft/2,nfft)
    dsampled = temp_vec*dt_sampled*c/2                    #depth vector sampled sampled fringe
    
    #FFT
    Ss0 = np.fft.fftshift(np.fft.ifft(ss,nfft))
    Ssout = Ss0
    Ss = np.square(np.abs(Ssout))
    
    if SNR:
        Ss_snr = np.fft.fftshift(np.fft.ifft(ss_snr,nfft))
        Ss_snr = np.square(np.abs(Ss_snr))
        #Ss_snr = Ss_snr/np.max(Ss) # normalize to PSF magnitude
        
        mag = np.max(Ss)
        magind = np.argmax(Ss)
        noise = np.mean(Ss_snr[magind-400:magind+90])
        snr = mag/noise
    else:
        snr = 0            
    
    return(sf, sfilt_plot, ss, Ss, dsampled, snr)
    